In [241]:
import numpy as np
import random
import math
import time
from shapely.geometry import Polygon, MultiPolygon, Point, box
from shapely.affinity import scale, translate
from shapely import wkt
from shapely.ops import transform
import pyproj
from functools import partial
import folium
from folium.features import DivIcon
from IPython.display import display, clear_output
from branca.element import Figure

In [242]:
def calculateArea(poly_wkt):
    polygon = wkt.loads(poly_wkt)
    # Method 1: Basic area calculation (in square degrees, not very useful)
    area_sq_degrees = polygon.area
    # print(f"Area (sq deg): {area_sq_degrees}")

    # Method 2: Calculate area using a proper projection for the location. Get the center point of the polygon to determine appropriate projection
    centroid = polygon.centroid
    lon, lat = centroid.x, centroid.y
    # Create a projection suitable for this region (UTM) Determine the UTM zone based on longitude
    utm_zone = int((lon + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    # Set up the transformation
    project = partial(
        pyproj.transform,
        pyproj.Proj("EPSG:4326"),  # source coordinate system (WGS84)
        pyproj.Proj(proj_string)   # destination coordinate system (UTM)
    )
    # Transform the polygon to the UTM projection
    utm_polygon = transform(project, polygon)
    area_sq_meters = utm_polygon.area
    area_sq_kilometers = area_sq_meters / 1_000_000  # Convert to square kilometers
    # area_hectares = area_sq_meters / 10000  # Convert to hectares
    # print(f"Area (m²): {area_sq_meters:.2f}")
    # print(f"Area (km²): {area_sq_kilometers:.2f}")
    # print(f"Area in hectares: {area_hectares:.2f} ha") 
    # print(f"UTM ZONE: {utm_zone}")
    return area_sq_kilometers


In [ ]:
def generate_random_polygon(bbox, num_points):
    """Generate a random polygon within the bounding box with the given number of points."""
    min_x, min_y, max_x, max_y = bbox
    width = max_x - min_x
    height = max_y - min_y
    
    # Generate random center point
    center_x = min_x + width / 2
    center_y = min_y + height / 2
    
    # Generate random points at varied distances around the center
    radius = min(width, height) / 2 * 0.8  # 80% of half the smallest dimension
    
    # Generate points around the center at varying distances
    points = []
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        # Vary the radius a bit
        r = radius * random.uniform(0.5, 1.0) # How uniform are the points (0.1 = random,  1.0 = uniform)
        x = center_x + r * math.cos(angle)
        y = center_y + r * math.sin(angle)
        points.append((x, y))
    
    # Close the polygon
    points.append(points[0])
    return points

def genCase1(bbox, num_points):
    """Case 1: Simple polygon with 3 or more points."""
    polygon = generate_random_polygon(bbox, num_points)
    return f"POLYGON (({' ,'.join([f'{x} {y}' for x, y in polygon])}))"

In [3]:
def create_polygon_with_area_percentage(bbox, percentage):
    """Create a polygon with a specific percentage of the bounding box area."""
    min_x, min_y, max_x, max_y = bbox
    width = max_x - min_x
    height = max_y - min_y
    
    # Calculate the scaling factor to achieve the desired area percentage
    scale_factor = math.sqrt(percentage)
    
    # Calculate the dimensions of the new polygon
    new_width = width * scale_factor
    new_height = height * scale_factor
    
    # Calculate the center of the bounding box
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    
    # Calculate the coordinates of the new polygon
    new_min_x = center_x - new_width / 2
    new_min_y = center_y - new_height / 2
    new_max_x = center_x + new_width / 2
    new_max_y = center_y + new_height / 2
    
    # Create the new polygon
    return [(new_min_x, new_min_y), (new_min_x, new_max_y), 
            (new_max_x, new_max_y), (new_max_x, new_min_y), (new_min_x, new_min_y)]

def genCase2(bbox, area_percentage=0.5):
    """Case 2: Simple polygon with varying area percentage of BBOX."""
    polygon = create_polygon_with_area_percentage(bbox, area_percentage)
    return f"POLYGON (({' ,'.join([f'{x} {y}' for x, y in polygon])}))"

In [ ]:
def genPolygon_AreaFIXED(INPUT_BBOX, num_points=None, area_percentage=None):
    """
    Generate different polygons within the INPUT_BBOX in EPSG:4326 - WGS 84
    Ranges from small area polygon to large polygons with large area in OGC WKT format
    
    Args:
        INPUT_BBOX: List [min_lon, min_lat, max_lon, max_lat]
        area_percentage: Optional float between 0 and 1, representing what percentage of the bbox area 
                        the polygon should occupy. If None, a random value is used.
        
    Returns:
        String: WKT format polygon
    """
    min_lon, min_lat, max_lon, max_lat = INPUT_BBOX
       
    # Get bbox dimensions
    bbox_width = max_lon - min_lon
    bbox_height = max_lat - min_lat
    
    # Calculate center of bbox
    center_lon = (min_lon + max_lon) / 2
    center_lat = (min_lat + max_lat) / 2

    # Calculate anywhere of bbox
    # center_lon = random.uniform(min_lon, max_lon)
    # center_lat = random.uniform(min_lat, max_lat)
    
    # Calculate polygon dimensions based on area percentage
    # For simplicity, we'll create a rectangular polygon
    scale_factor = math.sqrt(area_percentage)
    poly_width = bbox_width * scale_factor
    poly_height = bbox_height * scale_factor
    
    # Calculate polygon corners
    half_width = poly_width / 2
    half_height = poly_height / 2
    
    # Create a more interesting polygon by adding some random points and perturbations
    points = []
    
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        
        # Base radius is determined by the desired area
        x_radius = half_width
        y_radius = half_height
        
        # Add some randomness but maintain approximate area
        radius_factor = random.uniform(0.5, 1.0) # How uniform are the points (0.1 = random,  1.0 = uniform)
        
        x = center_lon + math.cos(angle) * x_radius * radius_factor
        y = center_lat + math.sin(angle) * y_radius * radius_factor
        
        # Ensure points are within the bbox
        x = max(min(x, max_lon), min_lon)
        y = max(min(y, max_lat), min_lat)
        
        points.append((x, y))
    
    # Close the polygon by adding the first point at the end
    points.append(points[0])
    
    # Convert to WKT format
    wkt = "POLYGON (("
    wkt += ", ".join([f"{p[0]} {p[1]}" for p in points])
    wkt += "))"

    # Double-check OGC compliance
    # is_valid, message = validate_ogc_polygon(wkt)
    # if not is_valid:
    #     # Try to fix the polygon
    #     wkt = fix_ogc_polygon(wkt)
    return wkt

In [74]:
def view_polygons_on_osm(i, polygon, bbox, area_percentage=None):
    # Create a map with the center based on the bounding box
    map_center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]
    m = folium.Map(location=map_center, zoom_start=10) # SWEDEN ZOOM 4
    
    if isinstance(polygon, str):
        # Assuming the string is WKT format
        try:
            shape = wkt.loads(polygon)
            geo_json = shape.__geo_interface__
        except Exception as e:
            print(f"Error parsing polygon {e}")
    else:
        # Assuming it's already in a format folium can use
        geo_json = polygon
        
        # Add the polygon to the map with a unique style
    folium.GeoJson(
        geo_json,
        style_function=lambda x, color=f'#4400ff': {
                'fillColor': color,
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
        }
    ).add_to(m)

    north, south, east, west = bbox[3], bbox[1], bbox[0], bbox[2]
    # Add the bounding box to the map
    folium.Rectangle(bounds=[[north, west], [south, east]], color="red", fill=False).add_to(m)

    folium.map.Marker(
        [north+0.1, east+0.3],
        icon=DivIcon(
            icon_size=(350,56),
            icon_anchor=(0,0),
            html='<div style="font-size: 18pt; color: red;">points '+str(i)+' | area '+str(area_percentage*100.0)+'% BBOX</div>',
            )
        ).add_to(m)
    display(m)
    clear_output(wait=True)
    time.sleep(0.5)  # Pause to see the update
    # Final display to keep the map visible after the loop
    # display(m)
    return m

In [236]:
## USER GLOBAL VARIABLES
# INPUT_BBOX = [11.015629, 55.128649, 24.199222, 69.380313] ## Sweden
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305]
NUM_OF_POINTS = 10  # 1,2 points are not polygon, so start with atleast 3 points 
NUM_OF_HOLES = 100
UNIFORM_FACTOR = 0.5 # How uniform are the points (0.1 = random,  1.0 = uniform)
PERCENTAGE_OF_AREA = 0.05

In [243]:
for i in range(3, NUM_OF_POINTS):
    polygon = genCase1(INPUT_BBOX, i)
    print(f"Points: {i} - {polygon}")
    view_polygons_on_osm(i, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

Points: 9 - POLYGON ((11.590259982551167 48.15486111114964, 11.578944300104771 48.18595069315683, 11.550291986342051 48.202493114220644, 11.517709777307585 48.19674799167679, 11.496443250719512 48.17140353221345, 11.496443250719512 48.13831869008583, 11.517709777307584 48.11297423062249, 11.550291986342051 48.107229108078634, 11.578944300104771 48.123771529142445, 11.590259982551167 48.15486111114964))


## BM 1 - POLYGON WITH FIXED AREA and VARYING NUMBER OF POINTS


In [210]:
## USER GLOBAL VARIABLES
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305] # MUNICH
NUM_OF_POINTS = 10
PERCENTAGE_OF_AREA = 0.25 # FIXED
UNIFORM_FACTOR = 0.5 # FIXED

In [211]:
for i in range(3, NUM_OF_POINTS):
    polygon = genPolygon_AreaFIXED(INPUT_BBOX, i, area_percentage=PERCENTAGE_OF_AREA)
    print(f"Points: {i} - {polygon}")
    view_polygons_on_osm(i, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

Points: 9 - POLYGON ((11.599773327113732 48.15486111114964, 11.607973188616322 48.183391344831875, 11.550720189359648 48.18048965792826, 11.503370876039064 48.18926513228166, 11.489787098391542 48.164636636685614, 11.496707921713373 48.14638271229324, 11.500325363452092 48.11774077717164, 11.554231189966796 48.11897909366692, 11.60837697205543 48.126156407454076, 11.599773327113732 48.15486111114964))


## BM 2 - POLYGON WITH FIXED POINTS and VARYING AREA

In [249]:
## USER GLOBAL VARIABLES
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305]
NUM_OF_POINTS = 5  # FIXED
# PERCENTAGE_OF_AREA = 0.05 # VARYING

In [ ]:
for PERCENTAGE_OF_AREA in np.arange(0.0, 1.02, 0.02):
    polygon = genPolygon_AreaFIXED(INPUT_BBOX, NUM_OF_POINTS, PERCENTAGE_OF_AREA)
    print(f"Area: {PERCENTAGE_OF_AREA*100.0:.2f}% - {(calculateArea(polygon)/calculateArea(bbox.wkt))*100.0:.2f} - {polygon}")
    view_polygons_on_osm(NUM_OF_POINTS, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

Area: 100.00% - 85.81 - POLYGON ((11.723194444453823 48.15486111114964, 11.636735893691737 48.24819444448305, 11.360694444453532 48.24819444448305, 11.360694444453532 48.06152777781623, 11.603789829024329 48.06152777781623, 11.723194444453823 48.15486111114964))


C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))


In [234]:
from shapely import wkt
import pyproj
from shapely.ops import transform
from functools import partial

# The WKT polygon string
polygon_wkt = "POLYGON((73.72558951377867 22.18740416353174,79.70215201377867 28.071979512891133,83.86230647563934 18.812717010101338,73.72558951377867 22.18740416353174))"
INPUT_BBOX_WKT = 'POLYGON ((11.723194444453823 48.06152777781623, 11.723194444453823 48.24819444448305, 11.360694444453532 48.24819444448305, 11.360694444453532 48.06152777781623, 11.723194444453823 48.06152777781623))'
# Parse the WKT string to a shapely geometry
polygon = wkt.loads(polygon_wkt)

# Method 1: Basic area calculation (in square degrees, not very useful)
area_sq_degrees = polygon.area
print(f"Area (sq deg): {area_sq_degrees}")

# Method 2: Calculate area using a proper projection for the location
# Get the center point of the polygon to determine appropriate projection
centroid = polygon.centroid
lon, lat = centroid.x, centroid.y

# Create a projection suitable for this region (UTM)
# Determine the UTM zone based on longitude
utm_zone = int((lon + 180) / 6) + 1
proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

# Set up the transformation
project = partial(
    pyproj.transform,
    pyproj.Proj("EPSG:4326"),  # source coordinate system (WGS84)
    pyproj.Proj(proj_string)   # destination coordinate system (UTM)
)

# Transform the polygon to the UTM projection
utm_polygon = transform(project, polygon)

# Calculate the area in square meters
area_sq_meters = utm_polygon.area
area_sq_kilometers = area_sq_meters / 1_000_000  # Convert to square kilometers
# area_hectares = area_sq_meters / 10000  # Convert to hectares

print(f"Area (m²): {area_sq_meters:.2f}")
print(f"Area (km²): {area_sq_kilometers:.2f}")
# print(f"Area in hectares: {area_hectares:.2f} ha")
print(f"UTM ZONE: {utm_zone}")

Area (sq deg): 39.90965172381122
Area (m²): 80809574223.21
Area (km²): 80809.57
UTM ZONE: 44


C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))


In [222]:
def calculateArea(poly_wkt):
    polygon = wkt.loads(poly_wkt)
    # Method 1: Basic area calculation (in square degrees, not very useful)
    area_sq_degrees = polygon.area
    # print(f"Area (sq deg): {area_sq_degrees}")

# Method 2: Calculate area using a proper projection for the location
# Get the center point of the polygon to determine appropriate projection
    centroid = polygon.centroid
    lon, lat = centroid.x, centroid.y

# Create a projection suitable for this region (UTM)
# Determine the UTM zone based on longitude
    utm_zone = int((lon + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

    # Set up the transformation
    project = partial(
        pyproj.transform,
        pyproj.Proj("EPSG:4326"),  # source coordinate system (WGS84)
        pyproj.Proj(proj_string)   # destination coordinate system (UTM)
    )

# Transform the polygon to the UTM projection
    utm_polygon = transform(project, polygon)

# Calculate the area in square meters
    area_sq_meters = utm_polygon.area
    area_sq_kilometers = area_sq_meters / 1_000_000  # Convert to square kilometers
# area_hectares = area_sq_meters / 10000  # Convert to hectares

    # print(f"Area (m²): {area_sq_meters:.2f}")
    # print(f"Area (km²): {area_sq_kilometers:.2f}")
    # print(f"Area in hectares: {area_hectares:.2f} ha") 
    # print(f"UTM ZONE: {utm_zone}")

    return area_sq_kilometers


In [227]:
input_poly = 'POLYGON((11.74324035644531 48.10926514749488,11.610031127929688 48.06186686091715,11.37702944688499 48.12790666405195,11.405410822480917 48.19843873103838,11.486892728134992 48.225587543508,11.706619318574667 48.20301538599938,11.74324035644531 48.10926514749488))'
calculateArea(input_poly)

C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))


807.7152880431923

In [195]:
bbox = shapely.geometry.box(*INPUT_BBOX, ccw=True)
bbox.wkt

'POLYGON ((11.723194444453823 48.06152777781623, 11.723194444453823 48.24819444448305, 11.360694444453532 48.24819444448305, 11.360694444453532 48.06152777781623, 11.723194444453823 48.06152777781623))'

In [196]:
calculateArea(bbox.wkt)

Area (sq deg): 0.06766666666677522
UTM ZONE: 32


C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))


1326.7485021089565

In [204]:
(calculateArea(input_poly)/calculateArea(bbox.wkt))*100.0

Area (sq deg): 0.0412050922877469
UTM ZONE: 32
Area (sq deg): 0.06766666666677522
UTM ZONE: 32


C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))


60.879306572366524

In [ ]:
# from shapely import wkt
# from pyproj import CRS, Transformer

# # WKT string of a polygon
# wkt_polygon = 'POLYGON((10.689 -25.092, 34.595 -20.170, 38.814 -35.639, 13.502 -39.155, 10.689 -25.092))'

# # Parse the WKT string into a Shapely geometry object
# polygon = wkt.loads(wkt_polygon)

# # Define the CRS for WGS84 and UTM (example UTM zone 38N, EPSG:32638)
# wgs84 = CRS('EPSG:4326')
# utm = CRS('EPSG:32638')

# # Create a transformer to convert between the two CRSs
# project = Transformer.from_crs(wgs84, utm, always_xy=True)

# Reproject the polygon
# projected_polygon = project.transform(polygon)

# # Calculate the area in square meters
# area = projected_polygon.area
# print(f"The area of the polygon is {area} square meters.")a

In [92]:
def get_bbox_polygon(bbox):
    minx, miny, maxx, maxy = bbox
    return box(minx, miny, maxx, maxy)

def create_regular_polygon(center_x, center_y, radius, num_points):
    """Generates a regular convex polygon"""
    return Polygon([
        (
            center_x + radius * math.cos(2 * math.pi * i / num_points),
            center_y + radius * math.sin(2 * math.pi * i / num_points)
        )
        for i in range(num_points)
    ])

def scale_to_bbox_area(geom, target_area):
    """Scales a geometry to match a target area"""
    current_area = geom.area
    scale_factor = math.sqrt(target_area / current_area)
    return scale(geom, xfact=scale_factor, yfact=scale_factor, origin='center')

# GENERATOR SUB FUNCTIONS

def genCase1(bbox, num_points, percentage_of_area=0.1):
    """Simple polygon with fixed area (50% of BBOX)"""
    bbox_poly = get_bbox_polygon(bbox)
    cx, cy = bbox_poly.centroid.x, bbox_poly.centroid.y
    radius = min(bbox_poly.bounds[2] - bbox_poly.bounds[0],
                 bbox_poly.bounds[3] - bbox_poly.bounds[1]) / 4
    poly = create_regular_polygon(cx, cy, radius, num_points)
    poly_scaled = scale_to_bbox_area(poly, bbox_poly.area * percentage_of_area)
    return poly_scaled.wkt

def genCase2(bbox, num_points=4, percentage_of_area=1.0):
    """Simple polygon with varying area (1%-100%)"""
    return genCase1(bbox, num_points, percentage_of_area)

In [238]:
for PERCENTAGE_OF_AREA in np.arange(0.0, 1.02, 0.05):
    polygon = genCase1(INPUT_BBOX, NUM_OF_POINTS, PERCENTAGE_OF_AREA)
    print(f"Area: {PERCENTAGE_OF_AREA*100.0:.2f}% - {(calculateArea(polygon)/calculateArea(bbox.wkt))*100.0:.2f} -{polygon}")
    view_polygons_on_osm(NUM_OF_POINTS, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

Area: 100.00% - 100.00 -POLYGON ((11.693681987202439 48.15486111114964, 11.664702695222118 48.24405020099647, 11.588833923847737 48.29917208994746, 11.495054965059616 48.29917208994746, 11.419186193685228 48.24405020099647, 11.390206901704914 48.15486111114964, 11.419186193685228 48.06567202130282, 11.495054965059616 48.010550132351824, 11.588833923847737 48.010550132351824, 11.664702695222118 48.06567202130282, 11.693681987202439 48.15486111114964))


C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\arkag\AppData\Roaming\Python\Python310\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))


## BM 3 - POLYGON WITH VARYING POINTS and VARYING AREA